In [ ]:
import re
calendar = pd.read_csv("investingEconomicCalendarNews.csv", index_col=0)
calendar = calendar[calendar["importance"] == "high"]
events = calendar["event"].apply(lambda event: event[:event.find('(')].strip() if event.find('(') != -1 else event)
cleaned_events = [
    re.sub(r'\b(German|BoC|BoE|BoJ|ECB|FED|RBA|RBNZ|SNB)\b', '', event, flags=re.IGNORECASE).strip()
    for event in events
]
calendar["event"] = cleaned_events
cleaned_events = set(cleaned_events)


In [ ]:
cleaned_events

{'10-Year Note Auction',
 '30-Year Bond Auction',
 'ADP Nonfarm Employment Change',
 'Average Hourly Earnings',
 'CB Consumer Confidence',
 'CPI',
 'Chicago PMI',
 'Core CPI',
 'Core PCE Price Index',
 'Core Retail Sales',
 'Crude Oil Inventories',
 'Deposit Facility Rate',
 'Durable Goods Orders',
 'Existing Home Sales',
 'GDP',
 'ISM Manufacturing PMI',
 'ISM Manufacturing Prices',
 'ISM Non-Manufacturing PMI',
 'ISM Non-Manufacturing Prices',
 'Initial Jobless Claims',
 'Interest Rate Decision',
 'JOLTS Job Openings',
 'Manufacturing PMI',
 'New Home Sales',
 'Nonfarm Payrolls',
 'PPI',
 'Philadelphia  Manufacturing Index',
 'Retail Sales',
 'S&P Global Services PMI',
 'S&P Global US Manufacturing PMI',
 'Unemployment Rate'}

In [ ]:
categories = {
    'positive_if_increase': [
        'ADP Nonfarm Employment Change','Average Hourly Earnings','Core Retail Sales',
        'Durable Goods Orders','Existing Home Sales','GDP','ISM Manufacturing PMI',
        'JOLTS Job Openings','Manufacturing PMI','New Home Sales','Nonfarm Payrolls',
        'Retail Sales','S&P Global Services PMI','S&P Global US Manufacturing PMI',
        'Interest Rate Decision', '10-Year Note Auction','30-Year Bond Auction', 'CB Consumer Confidence','PPI',
        'CPI','Chicago PMI', 'Core CPI', 'Core PCE Price Index', 'ISM Non-Manufacturing PMI', "Philadelphia  Manufacturing Index", 'Deposit Facility Rate',
        'ISM Manufacturing Prices','ISM Non-Manufacturing Prices'
    ],
    'negative_if_increase': [
      'Crude Oil Inventories','Initial Jobless Claims',
      'Unemployment Rate'
    ]
}

In [ ]:
calendar["impact"] = [1 if event in categories['positive_if_increase'] else 0 for event in calendar["event"]]

In [ ]:
calendar

,id,date,time,zone,currency,importance,event,actual,forecast,previous,impact
7,485712,30/10/2023,09:00,germany,EUR,high,GDP,-0.1%,-0.3%,0.1%,1
8,484939,30/10/2023,13:00,germany,EUR,high,CPI,0.0%,0.2%,0.3%,1
12,484361,31/10/2023,01:30,china,CNY,high,Manufacturing PMI,49.5,50.2,50.2,1
14,484819,31/10/2023,03:30,japan,JPY,high,Interest Rate Decision,-0.10%,-0.10%,-0.10%,1
22,484923,31/10/2023,10:00,euro zone,EUR,high,CPI,2.9%,3.1%,4.3%,1
...,...,...,...,...,...,...,...,...,...,...,...
4893,509663,24/10/2024,12:30,united states,USD,high,Initial Jobless Claims,227K,243K,242K,0
4894,508987,24/10/2024,13:45,united states,USD,high,S&P Global US Manufacturing PMI,47.8,47.5,47.3,1
4896,508988,24/10/2024,13:45,united states,USD,high,S&P Global Services PMI,55.3,55.0,55.2,1
4898,508990,24/10/2024,14:00,united states,USD,high,New Home Sales,738K,719K,709K,1


In [ ]:
def classify_impact(row):
    baseline = row['forecast'] if not pd.isna(row['forecast']) else row['previous']

    if row["impact"] == 1:
        if row['actual'] > baseline:
            return 'positive'
        elif row['actual'] < baseline:
            return 'negative'
    else:
        if row['actual'] > baseline:
            return 'negative'
        elif row['actual'] < baseline:
            return 'positive'
    return 'neutral'

In [ ]:
calendar["sentiment"] = calendar.apply(classify_impact, axis=1)

In [ ]:
calendar.drop(columns = "impact", inplace = True)

In [ ]:
calendar

,id,date,time,zone,currency,importance,event,actual,forecast,previous,sentiment
7,485712,30/10/2023,09:00,germany,EUR,high,GDP,-0.1%,-0.3%,0.1%,negative
8,484939,30/10/2023,13:00,germany,EUR,high,CPI,0.0%,0.2%,0.3%,negative
12,484361,31/10/2023,01:30,china,CNY,high,Manufacturing PMI,49.5,50.2,50.2,negative
14,484819,31/10/2023,03:30,japan,JPY,high,Interest Rate Decision,-0.10%,-0.10%,-0.10%,neutral
22,484923,31/10/2023,10:00,euro zone,EUR,high,CPI,2.9%,3.1%,4.3%,negative
...,...,...,...,...,...,...,...,...,...,...,...
4893,509663,24/10/2024,12:30,united states,USD,high,Initial Jobless Claims,227K,243K,242K,positive
4894,508987,24/10/2024,13:45,united states,USD,high,S&P Global US Manufacturing PMI,47.8,47.5,47.3,positive
4896,508988,24/10/2024,13:45,united states,USD,high,S&P Global Services PMI,55.3,55.0,55.2,positive
4898,508990,24/10/2024,14:00,united states,USD,high,New Home Sales,738K,719K,709K,positive


In [ ]:
import pandas as pd
news = pd.read_csv("topCurrenciesUSDFirstInvestingComNews.csv", index_col=0)
news

,pair,title,description,link,provider,time_published
0,usd-jpy,"Asian stocks mixed, dollar drifts before megac...",By Kevin Buckland TOKYO (Reuters) -Asian stock...,https://www.investing.com/news/economy-news/as...,Reuters,2024-10-29 07:00:43
1,usd-jpy,Yen mired in political uncertainty; dollar fir...,By Rae Wee SINGAPORE (Reuters) -The yen langui...,https://www.investing.com/news/economy-news/ye...,Reuters,2024-10-29 06:20:33
2,usd-jpy,Asia FX softens with econ. data in focus; yen ...,Investing.com-- Most Asian currencies weakened...,https://www.investing.com/news/forex-news/asia...,Investing.com,2024-10-29 03:50:36
3,usd-jpy,Japan's finance minister vows to closely watch...,TOKYO (Reuters) - Japan's Finance Minister Kat...,https://www.investing.com/news/economy-news/ja...,Reuters,2024-10-29 03:05:32
4,usd-jpy,Asia FX at risk as dollar firms into US electi...,Investing.com-- Asian currencies are at risk o...,https://www.investing.com/news/forex-news/asia...,Investing.com,2024-10-29 00:09:28
...,...,...,...,...,...,...
3974,usd-idr,Indonesia's Q3 growth falls short of expectati...,"Indonesia, Southeast Asia's largest economy, p...",https://www.investing.com/news/economy/indones...,Investing.com,2023-11-06 09:20:41
3975,usd-idr,Asia FX firms as dollar extends losses before ...,Investing.com-- Most Asian currencies advanced...,https://www.investing.com/news/forex-news/asia...,Investing.com,2023-11-03 04:14:41
3976,usd-idr,"Exclusive-EU, US, COP28 hosts rally support fo...",By Kate Abnett and Valerie Volcovici (Reuters)...,https://www.investing.com/news/commodities-new...,Reuters,2023-11-01 15:18:53
3977,usd-idr,"Indonesia sets emission target for G7 funding,...",By Fransiska Nangoy JAKARTA (Reuters) -Indones...,https://www.investing.com/news/commodities-new...,Reuters,2023-11-01 06:30:41


In [ ]:
from transformers import pipeline, BertForSequenceClassification, BertTokenizer
modelBERT = BertForSequenceClassification.from_pretrained("ProsusAI/finbert", num_labels=3)
tokenizerBERT = BertTokenizer.from_pretrained("ProsusAI/finbert")

def fin_BERT(text):
  nlp = pipeline("sentiment-analysis", model=modelBERT, tokenizer=tokenizerBERT, device='cuda')
  return nlp(text)[0]["label"]

In [ ]:
def sentiment_analyse(row):
  sentiment_title = fin_BERT(row["title"])
  sentiment_description = fin_BERT(row["description"])
  if sentiment_title == sentiment_description:
    return sentiment_title
  return "neutral"

In [ ]:
news["sentiment"] = news.apply(sentiment_analyse, axis=1)

In [ ]:
news.head(30)

,pair,title,description,link,provider,time_published,sentiment
0,usd-jpy,"Asian stocks mixed, dollar drifts before megac...",By Kevin Buckland TOKYO (Reuters) -Asian stock...,https://www.investing.com/news/economy-news/as...,Reuters,2024-10-29 07:00:43,negative
1,usd-jpy,Yen mired in political uncertainty; dollar fir...,By Rae Wee SINGAPORE (Reuters) -The yen langui...,https://www.investing.com/news/economy-news/ye...,Reuters,2024-10-29 06:20:33,negative
2,usd-jpy,Asia FX softens with econ. data in focus; yen ...,Investing.com-- Most Asian currencies weakened...,https://www.investing.com/news/forex-news/asia...,Investing.com,2024-10-29 03:50:36,neutral
3,usd-jpy,Japan's finance minister vows to closely watch...,TOKYO (Reuters) - Japan's Finance Minister Kat...,https://www.investing.com/news/economy-news/ja...,Reuters,2024-10-29 03:05:32,neutral
4,usd-jpy,Asia FX at risk as dollar firms into US electi...,Investing.com-- Asian currencies are at risk o...,https://www.investing.com/news/forex-news/asia...,Investing.com,2024-10-29 00:09:28,negative
5,usd-jpy,"Stocks gain, oil falls 6%; big tech-related ea...",By Caroline Valetkevitch NEW YORK (Reuters) -G...,https://www.investing.com/news/economy-news/oi...,Reuters,2024-10-28 21:56:39,positive
6,usd-jpy,"Morning Bid: Uncertainty runs deep, investors ...",By Jamie McGeever (Reuters) - A look at the da...,https://www.investing.com/news/economy-news/mo...,Reuters,2024-10-28 21:51:34,neutral
7,usd-jpy,Japan's government in flux after election give...,"By John Geddie, Tim Kelly and Sakura Murakami ...",https://www.investing.com/news/economy-news/ja...,Reuters,2024-10-28 16:03:07,neutral
8,usd-jpy,Japan stocks rally as yen tumbles after coalit...,By Brigid Riley and Kevin Buckland TOKYO (Reut...,https://www.investing.com/news/economy-news/ja...,Reuters,2024-10-28 16:01:54,negative
9,usd-jpy,"Japan's political shakeup complicates BOJ, fis...",By Makiko Yamazaki TOKYO (Reuters) - The loss ...,https://www.investing.com/news/economy-news/ja...,Reuters,2024-10-28 15:35:44,neutral
